<a href="https://colab.research.google.com/github/openUC2/UC2-REST/blob/master/DOCUMENTATION/DOC_UC2Client.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

# UC2 REST Tutorial

Here we are going to teach you how to interact with the UC2 microcontroller and how you can add additional functionalities. 

In order to use the client in your python environment you need the following packages to be installed:

(use the `!` to install it from within this jupyter notebook)
```py
!pip install UC2-REST
```

This code has been tested with the ESP32 WEMOS D1 R32 + CNC shield v3, where 3 stepper are connected to the board and an LED Matrix (WS2812, adafruit) is connected to the FEED pin. 

If you find an error, please report it [here](https://github.com/openUC2/UC2-REST/issues/new) 

In [1]:
!pip install UC2-REST # only if you have not installed it already
!pip install progressbar2

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.

## Organize all imports

First of all we need to import the `ESP32Client`. Since it is not yet a standalone pip package, we have to do that via a relaitve import, meaning that the file is in the same folder as this Jupyter Notebook

In [1]:
%load_ext autoreload 
%autoreload 2


import uc2rest
import time
import numpy as np

/opt/anaconda3/envs/imswitch39/lib/python3.9/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(


# Connecting to the ESP32 via USB

Now we want to initiliaze the USB-serial connection. Therefore, connect the ESP32 to your computer using a data (!) USB cable and establish the connection. You can leave the port as "unknown" as the portfinder may identify the ESP.

**Important:** Close all applications that may be connected to the ESP (e.g. Arduino Serial Plotter)

**IMPORTANT:** Install the USB serial driver for the ESP32: https://learn.sparkfun.com/tutorials/how-to-install-ch340-drivers/all

In [2]:
ESP32 = uc2rest.UC2Client(serialport="unknown")

# setting debug output of the serial to true - all message will be printed
ESP32.serial.DEBUG=True

2022-11-23 08:42:27 ERROR [UC2Client] [Errno 2] could not open port unknown: [Errno 2] No such file or directory: 'unknown'
2022-11-23 08:42:27 DEBUG [UC2Client] /dev/cu.usbserial-0001
2022-11-23 08:42:30 DEBUG [UC2Client] {"task": "/state_get"}
2022-11-23 08:42:30 DEBUG [UC2Client] We are connected: True on port: /dev/cu.usbserial-0001
2022-11-23 08:42:30 DEBUG [UC2Client] Attention, lasers are on channels 1,2,3
2022-11-23 08:42:30 DEBUG [UC2Client] {"task": "/motor_get"}
2022-11-23 08:42:30 DEBUG [UC2Client] {"task": "/ledarr_get"}
2022-11-23 08:42:30 DEBUG [UC2Client] {"task": "/laser_get"}
2022-11-23 08:42:30 DEBUG [UC2Client] {"task": "/state_get"}


loaded config is valid


# Test the REST

We can test the serial and check which firmware was uploaded when:

In [5]:
# get motor response
test_cmd = "{'task': '/motor_get'}"
ESP32.serial.writeSerial(test_cmd)
cmd_return = ESP32.serial.readSerial()
print(cmd_return)

# get state response
test_cmd = "{'task': '/state_get'}"
ESP32.serial.writeSerial(test_cmd)
cmd_return = ESP32.serial.readSerial()
print(cmd_return)





2022-11-23 08:44:51 DEBUG [UC2Client] {'task': '/motor_get'}
2022-11-23 08:44:52 DEBUG [UC2Client] {'task': '/state_get'}


{'steppers': [{'stepperid': 0, 'dir': 14, 'step': 17, 'enable': 12, 'dir_inverted': False, 'step_inverted': False, 'enable_inverted': False, 'speed': 0, 'speedmax': 200000, 'max_pos': 100000, 'min_pos': -100000, 'position': 0, 'isbusy': False}, {'stepperid': 1, 'dir': 16, 'step': 26, 'enable': 12, 'dir_inverted': False, 'step_inverted': False, 'enable_inverted': False, 'speed': 0, 'speedmax': 200000, 'max_pos': 100000, 'min_pos': -100000, 'position': 0, 'isbusy': False}, {'stepperid': 2, 'dir': 27, 'step': 25, 'enable': 12, 'dir_inverted': False, 'step_inverted': False, 'enable_inverted': False, 'speed': 0, 'speedmax': 200000, 'max_pos': 100000, 'min_pos': -100000, 'position': 0, 'isbusy': False}, {'stepperid': 3, 'dir': 0, 'step': 0, 'enable': 0, 'dir_inverted': False, 'step_inverted': False, 'enable_inverted': False, 'speed': 0, 'speedmax': 200000, 'max_pos': 100000, 'min_pos': -100000, 'position': 0, 'isbusy': False}]}
{'identifier_name': 'UC2_Feather', 'identifier_id': 'V2.0', 'ide

# Moving the motor 

The following code snippets will help you moving the motors (XYZ) continously or at a known number of `steps` at a certain `speed` level (all measured in steps/s). 

The additional attributs 
- `is_blocking` states if the action is performed in the background or not; if `False` no return message will be provided
- `is_absolute` says if we go relative or absolute steps 
- `is_enabled` says if we want to "unpower" the motors once we are done (prevent overheating)


Once the microcontroller (ESP32) is flashed, the pin definitions are not set yet. We have to do that manually using a customized dictionary and write that to the MCU.

In [7]:
# setup all motors at once 
print(ESP32.motor.settingsdict)

# Remember => 0 = A, 1 = X, 2 = Y, 3 = Z w.r.t. the axises of the microscope

ESP32.motor.settingsdict["motor"]["steppers"][0]["dir"]=18
ESP32.motor.settingsdict["motor"]["steppers"][0]["step"]=19
ESP32.motor.settingsdict["motor"]["steppers"][0]["enable"]=12

ESP32.motor.settingsdict["motor"]["steppers"][1]["dir"]=16
ESP32.motor.settingsdict["motor"]["steppers"][1]["step"]=26
ESP32.motor.settingsdict["motor"]["steppers"][2]["dir"]=27

ESP32.motor.settingsdict["motor"]["steppers"][2]["step"]=25
ESP32.motor.settingsdict["motor"]["steppers"][3]["dir"]=14
ESP32.motor.settingsdict["motor"]["steppers"][3]["step"]=17

ESP32.motor.settingsdict["motor"]["steppers"][1]["enable"]=12
ESP32.motor.settingsdict["motor"]["steppers"][2]["enable"]=12
ESP32.motor.settingsdict["motor"]["steppers"][3]["enable"]=12
ESP32.motor.set_motors(ESP32.motor.settingsdict)

# check if we set the right parameters
ESP32.motor.get_motors()


2022-11-23 08:46:40 DEBUG [UC2Client] {"motor": {"steppers": [{"stepperid": 0, "dir": 18, "step": 19, "enable": 12, "dir_inverted": false, "step_inverted": false, "enable_inverted": false, "speed": 0, "speedmax": 200000, "max_pos": 100000, "min_pos": -100000}, {"stepperid": 1, "dir": 16, "step": 26, "enable": 12, "dir_inverted": false, "step_inverted": false, "enable_inverted": false, "speed": 0, "speedmax": 200000, "max_pos": 100000, "min_pos": -100000}, {"stepperid": 2, "dir": 27, "step": 25, "enable": 12, "dir_inverted": false, "step_inverted": false, "enable_inverted": false, "speed": 0, "speedmax": 200000, "max_pos": 100000, "min_pos": -100000}, {"stepperid": 3, "dir": 14, "step": 17, "enable": 12, "dir_inverted": false, "step_inverted": false, "enable_inverted": false, "speed": 0, "speedmax": 200000, "max_pos": 100000, "min_pos": -100000}]}, "task": "/motor_set"}


{'motor': {'steppers': [{'stepperid': 0, 'dir': 18, 'step': 19, 'enable': 12, 'dir_inverted': False, 'step_inverted': False, 'enable_inverted': False, 'speed': 0, 'speedmax': 200000, 'max_pos': 100000, 'min_pos': -100000}, {'stepperid': 1, 'dir': 16, 'step': 26, 'enable': 12, 'dir_inverted': False, 'step_inverted': False, 'enable_inverted': False, 'speed': 0, 'speedmax': 200000, 'max_pos': 100000, 'min_pos': -100000}, {'stepperid': 2, 'dir': 27, 'step': 25, 'enable': 12, 'dir_inverted': False, 'step_inverted': False, 'enable_inverted': False, 'speed': 0, 'speedmax': 200000, 'max_pos': 100000, 'min_pos': -100000}, {'stepperid': 3, 'dir': 14, 'step': 17, 'enable': 12, 'dir_inverted': False, 'step_inverted': False, 'enable_inverted': False, 'speed': 0, 'speedmax': 200000, 'max_pos': 100000, 'min_pos': -100000}]}, 'task': '/motor_set'}


2022-11-23 08:46:40 DEBUG [UC2Client] {"task": "/motor_get", "isbusy": 1}
2022-11-23 08:46:40 DEBUG [UC2Client] {"task": "/motor_get"}


[{'stepperid': 0,
  'dir': 18,
  'step': 19,
  'enable': 12,
  'dir_inverted': False,
  'step_inverted': False,
  'enable_inverted': False,
  'speed': 0,
  'speedmax': 200000,
  'max_pos': 100000,
  'min_pos': -100000,
  'position': 0,
  'isbusy': False},
 {'stepperid': 1,
  'dir': 16,
  'step': 26,
  'enable': 12,
  'dir_inverted': False,
  'step_inverted': False,
  'enable_inverted': False,
  'speed': 0,
  'speedmax': 200000,
  'max_pos': 100000,
  'min_pos': -100000,
  'position': 0,
  'isbusy': False},
 {'stepperid': 2,
  'dir': 27,
  'step': 25,
  'enable': 12,
  'dir_inverted': False,
  'step_inverted': False,
  'enable_inverted': False,
  'speed': 0,
  'speedmax': 200000,
  'max_pos': 100000,
  'min_pos': -100000,
  'position': 0,
  'isbusy': False},
 {'stepperid': 3,
  'dir': 14,
  'step': 17,
  'enable': 12,
  'dir_inverted': False,
  'step_inverted': False,
  'enable_inverted': False,
  'speed': 0,
  'speedmax': 200000,
  'max_pos': 100000,
  'min_pos': -100000,
  'position':

In [8]:
# OR setup motors individually (according to WEMOS R32 D1)
ESP32.motor.set_motor(stepperid = 1, position = 0, stepPin = 26, dirPin=16, enablePin=12, maxPos=None, minPos=None, acceleration=None, isEnable=1)
ESP32.motor.set_motor(stepperid = 2, position = 0, stepPin = 25, dirPin=27, enablePin=12, maxPos=None, minPos=None, acceleration=None, isEnable=1)
ESP32.motor.set_motor(stepperid = 3, position = 0, stepPin = 17, dirPin=14, enablePin=12, maxPos=None, minPos=None, acceleration=None, isEnable=1)
ESP32.motor.set_motor(stepperid = 0, position = 0, stepPin = 19, dirPin=18, enablePin=12, maxPos=None, minPos=None, acceleration=None, isEnable=1)

# get individual motors
print(ESP32.motor.get_motor(axis = 1))


2022-11-23 08:48:16 DEBUG [UC2Client] {"task": "/motor_set", "motor": {"steppers": [{"stepperid": 1, "step": 26, "dir": 16, "enable": 12, "position": 0, "isen": 1}]}}
2022-11-23 08:48:16 DEBUG [UC2Client] {"task": "/motor_get", "isbusy": 1}
2022-11-23 08:48:16 DEBUG [UC2Client] {"task": "/motor_set", "motor": {"steppers": [{"stepperid": 2, "step": 25, "dir": 27, "enable": 12, "position": 0, "isen": 1}]}}
2022-11-23 08:48:17 DEBUG [UC2Client] {"task": "/motor_get", "isbusy": 1}
2022-11-23 08:48:17 DEBUG [UC2Client] {"task": "/motor_set", "motor": {"steppers": [{"stepperid": 3, "step": 17, "dir": 14, "enable": 12, "position": 0, "isen": 1}]}}
2022-11-23 08:48:17 DEBUG [UC2Client] {"task": "/motor_get", "isbusy": 1}
2022-11-23 08:48:17 DEBUG [UC2Client] {"task": "/motor_set", "motor": {"steppers": [{"stepperid": 0, "step": 19, "dir": 18, "enable": 12, "position": 0, "isen": 1}]}}
2022-11-23 08:48:17 DEBUG [UC2Client] {"task": "/motor_get", "isbusy": 1}
2022-11-23 08:48:17 DEBUG [UC2Client

{'stepperid': 1, 'dir': 16, 'step': 26, 'enable': 12, 'dir_inverted': False, 'step_inverted': False, 'enable_inverted': False, 'speed': 0, 'speedmax': 200000, 'max_pos': 100000, 'min_pos': -100000, 'position': 0, 'isbusy': False}


We can also setup individual motor settings like so:

In [9]:
ESP32.motor.set_motor_currentPosition(axis=0, currentPosition=10000)
ESP32.motor.set_motor_acceleration(axis=0, acceleration=10000)
ESP32.motor.set_motor_enable(is_enable=1)
ESP32.motor.set_direction(axis=1, sign=1, timeout=1)
ESP32.motor.set_position(axis=1, position=0, timeout=1)

# wait to settle
time.sleep(2)

2022-11-23 08:48:49 DEBUG [UC2Client] {"task": "/motor_set", "motor": {"steppers": [{"stepperid": 0, "position": 10000}]}}
2022-11-23 08:48:49 DEBUG [UC2Client] {"task": "/motor_get", "isbusy": 1}
2022-11-23 08:48:49 DEBUG [UC2Client] {"task": "/motor_set", "motor": {"steppers": [{"stepperid": 0, "acceleration": 10000}]}}
2022-11-23 08:48:49 DEBUG [UC2Client] {"task": "/motor_get", "isbusy": 1}
2022-11-23 08:48:49 DEBUG [UC2Client] {"task": "/motor_set", "motor": {"steppers": [{"stepperid": 0, "isen": 1}]}}
2022-11-23 08:48:49 DEBUG [UC2Client] {"task": "/motor_get", "isbusy": 1}


## Moving the motors

With the below code, we can test the motors spinning or not

In [10]:
# test Motor
position1 = ESP32.motor.get_position(timeout=1)
print(position1)
ESP32.motor.move_x(steps=10000, speed=10000, is_blocking=True)
ESP32.motor.move_y(steps=1000, speed=1000, is_blocking=True)
ESP32.motor.move_z(steps=1000, speed=1000, is_blocking=True)
ESP32.motor.move_t(steps=1000, speed=1000)
ESP32.motor.move_xyzt(steps=(0,10000,10000,0), speed=10000, is_blocking=True)
ESP32.motor.move_xyzt(steps=(0,0,0,0), speed=10000, is_absolute=True, is_blocking=True)
ESP32.motor.move_forever(speed=(0,100,0,0), is_stop=False)
time.sleep(1)
ESP32.motor.move_forever(speed=(0,0,0,0), is_stop=True)

position2 = ESP32.motor.get_position(timeout=1)
print(position2)


2022-11-23 08:49:33 DEBUG [UC2Client] {"task": "/motor_get", "position": 1}
2022-11-23 08:49:33 DEBUG [UC2Client] {"task": "/motor_act", "motor": {"steppers": [{"stepperid": 0, "position": 0, "speed": 0.0, "isabs": false, "isaccel": 0}, {"stepperid": 1, "position": 10000, "speed": 10000.0, "isabs": false, "isaccel": 0}, {"stepperid": 2, "position": 0, "speed": 0.0, "isabs": false, "isaccel": 0}, {"stepperid": 3, "position": 0, "speed": 0.0, "isabs": false, "isaccel": 0}]}}


[10000     0     0     0]


2022-11-23 08:49:34 DEBUG [UC2Client] {"task": "/motor_act", "motor": {"steppers": [{"stepperid": 0, "position": 0, "speed": 0.0, "isabs": false, "isaccel": 0}, {"stepperid": 1, "position": 0, "speed": 0.0, "isabs": false, "isaccel": 0}, {"stepperid": 2, "position": 1000, "speed": 1000.0, "isabs": false, "isaccel": 0}, {"stepperid": 3, "position": 0, "speed": 0.0, "isabs": false, "isaccel": 0}]}}
2022-11-23 08:49:35 DEBUG [UC2Client] {"task": "/motor_act", "motor": {"steppers": [{"stepperid": 0, "position": 0, "speed": 0.0, "isabs": false, "isaccel": 0}, {"stepperid": 1, "position": 0, "speed": 0.0, "isabs": false, "isaccel": 0}, {"stepperid": 2, "position": 0, "speed": 0.0, "isabs": false, "isaccel": 0}, {"stepperid": 3, "position": 1000, "speed": 1000.0, "isabs": false, "isaccel": 0}]}}
2022-11-23 08:49:36 DEBUG [UC2Client] {"task": "/motor_act", "motor": {"steppers": [{"stepperid": 0, "position": 1000, "speed": 1000.0, "isabs": false, "isaccel": 0}, {"stepperid": 1, "position": 0, "

[ 8424 18350  9350     0]


# ESP32 State

In [11]:
# test state
_state = ESP32.state.get_state()
print(_state)
ESP32.state.set_state(debug=False)
_mode = ESP32.state.isControllerMode()
print(_mode)
ESP32.state.espRestart() # restarts the microcontroller
time.sleep(5)
ESP32.state.setControllerMode(isController=True)
_busy = ESP32.state.isBusy()
print(_busy)

2022-11-23 08:50:09 DEBUG [UC2Client] {"task": "/state_get"}
2022-11-23 08:50:09 DEBUG [UC2Client] {"task": "/state_set", "isdebug": 0}
2022-11-23 08:50:09 DEBUG [UC2Client] {"task": "/state_get", "pscontroller": 1}
2022-11-23 08:50:09 DEBUG [UC2Client] {"restart": 1, "task": "/state_act"}


{'identifier_name': 'UC2_Feather', 'identifier_id': 'V2.0', 'identifier_date': 'Nov 22 202211:32:42', 'identifier_author': 'BD', 'IDENTIFIER_NAME': 'uc2-esp', 'configIsSet': False}
0


2022-11-23 08:50:16 DEBUG [UC2Client] {"task": "/state_act", "pscontroller": true}
2022-11-23 08:50:16 DEBUG [UC2Client] {"task": "/state_get", "isBusy": 1}


False


# LED Matrix

If the LED matrix is connected to pin 4 and has 16 individual WS2810 LEds, you can set it up like so

In [12]:
# setup led configuration
ESP32.led.set_ledpin(ledArrPin=4, ledArrNum=16)
print(ESP32.led.get_ledpin())

2022-11-23 08:51:36 DEBUG [UC2Client] {"task": "/ledarr_set", "ledArrPin": 4, "ledArrNum": 16}
2022-11-23 08:51:36 DEBUG [UC2Client] {"task": "/ledarr_get"}


{'ledArrNum': 0, 'ledArrPin': 0, 'LEDArrMode': [0, 1, 2, 3, 4, 5, 6, 7], 'led_ison': False}


In [13]:
# test LED
led_pattern = np.zeros((1, 5, 5, 3), dtype=np.uint8)
ESP32.led.send_LEDMatrix_array(led_pattern=led_pattern, timeout=1)
ESP32.led.send_LEDMatrix_full(intensity=(255, 0, 0), timeout=1)
ESP32.led.send_LEDMatrix_single(indexled=0, intensity=(0, 255, 0), timeout=1)

2022-11-23 08:51:39 DEBUG [UC2Client] Setting LED Pattern (array) 
2022-11-23 08:51:39 DEBUG [UC2Client] {"red": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], "green": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], "blue": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], "arraySize": 5, "LEDArrMode": "array", "task": "/ledarr_act"}
2022-11-23 08:51:39 DEBUG [UC2Client] Setting LED Pattern (full): (255, 0, 0)
2022-11-23 08:51:39 DEBUG [UC2Client] {"task": "/ledarr_act", "red": 255, "green": 0, "blue": 0, "LEDArrMode": "full"}
2022-11-23 08:51:39 DEBUG [UC2Client] Setting LED PAttern: 0 - (0, 255, 0)
2022-11-23 08:51:39 DEBUG [UC2Client] {"red": 0, "green": 255, "blue": 0, "indexled": 0, "LEDArrMode": "single", "task": "/ledarr_act"}


# Wifi
This feature will be implemented soon. It will help you to connect to a common Wifi hotspot or create an access point 

In [14]:
ESP32.wifi.scanWifi()

2022-11-23 08:52:24 DEBUG [UC2Client] {"task": "/wifi/scan"}


# Lasers

Lasers are essentially PWM pins and can tune a voltage signal between 0..3.3V in case of the ESP32
You can also hook up LEDs 

In [15]:
# set laser pins 
ESP32.laser.set_laserpin(laserid=1, laserpin=15)
ESP32.laser.set_laserpin(laserid=2, laserpin=16)
ESP32.laser.set_laserpin(laserid=3, laserpin=17)


2022-11-23 08:54:56 DEBUG [UC2Client] {"task": "/laser_set", "LASERid": 1, "LASERpin": 15}
2022-11-23 08:54:56 DEBUG [UC2Client] {"task": "/laser_set", "LASERid": 2, "LASERpin": 16}
2022-11-23 08:54:56 DEBUG [UC2Client] {"task": "/laser_set", "LASERid": 3, "LASERpin": 17}


In [16]:
# set laser values
ESP32.laser.set_laser(channel=1, value=1000, despeckleAmplitude=0, despecklePeriod=10, timeout=20, is_blocking = True)
ESP32.laser.set_laser(channel=2, value=1000, despeckleAmplitude=0, despecklePeriod=10, timeout=20, is_blocking = True)
ESP32.laser.set_laser(channel=3, value=1000, despeckleAmplitude=0, despecklePeriod=10, timeout=20, is_blocking = True)


2022-11-23 08:55:24 DEBUG [UC2Client] Setting Laser 1, value: 1000
2022-11-23 08:55:24 DEBUG [UC2Client] {"task": "/laser_act", "LASERid": 1, "LASERval": 1000, "LASERdespeckle": 0, "LASERdespecklePeriod": 10}
2022-11-23 08:55:24 DEBUG [UC2Client] Setting Laser 2, value: 1000
2022-11-23 08:55:24 DEBUG [UC2Client] {"task": "/laser_act", "LASERid": 2, "LASERval": 1000, "LASERdespeckle": 0, "LASERdespecklePeriod": 10}
2022-11-23 08:55:24 DEBUG [UC2Client] Setting Laser 3, value: 1000
2022-11-23 08:55:24 DEBUG [UC2Client] {"task": "/laser_act", "LASERid": 3, "LASERval": 1000, "LASERdespeckle": 0, "LASERdespecklePeriod": 10}


{'return': 1}